# Imports

In [138]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance 


# Input

In [139]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [140]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [141]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [142]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [143]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.70125446 0.70166246 0.70125446 ... 0.65284429 0.94765668 0.65284429]
[0.65548527 0.65586664 0.65548527 ... 0.61023472 0.88580542 0.61023471]
[0.59452687 0.59458054 0.59452687 ... 0.58814237 0.6264895  0.58814237]
(5999,)


### Multinomial Sampling

In [144]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[1 0 0 ... 0 0 0]


In [145]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[76.761116 ,  3.5589852, 97.95591  ],
       [79.10776  ,  7.6759295, 83.02889  ],
       [82.76066  ,  2.9401166, 96.43115  ],
       [78.63775  ,  7.6759295, 83.84298  ],
       [75.41062  ,  7.328478 , 85.10679  ]], dtype=float32)

# KNN extended graph

In [146]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [147]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11786


# Edge Predictor

In [148]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [149]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [150]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [151]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [152]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  (1, 11)	1
  :	:
  (1498, 1348)	1
  (1498, 1478)	1
  (1498, 1486)	1
  (1498, 1491)	1
  (1498, 1496)	1
  (1498, 1497)	1
  (1499, 1214)	1
  (1499, 1280)	1
  (1499, 1381)	1
  (1499, 1382)	1
  (1499, 1383)	1
  (1499, 1394)	1
  (1499, 1468)	1
  (1499, 1476)	1
  (1499, 1477)	1
  (1499, 1479)	1
  (1499, 1480)	1
  (1499, 1481)	1
  (1499, 1484)	1
  (1499, 1488)	1
  (1499, 1490)	1
  (1499, 1492)	1
  (1499, 1493)	1
  (1499, 1494)	1
  (1499, 1495)	1
[[0.05660038 0.05194451 0.05669836 ... 0.04116098 0.03860379 0.01454568]
 [1.98410606 0.05559632 0.05994631 ... 0.04514368 0.04257231 0.01702677]
 [1.87455159 0.51531543 0.05661953 ... 0.04106632 0.03850999 0.01449153]
 ...
 [2.2608784  2.27210524 2.26113626 ... 0.06381386 0.06387395 0.

In [153]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [154]:
print(A_s)  # symmétrique
print(A_s.shape)

[[2.86732595e-26 3.05301455e-26 2.93765604e-26 ... 1.05233040e-24
  1.17388485e-24 3.54500534e-24]
 [3.05301455e-26 3.25317447e-26 3.16934938e-26 ... 1.14690617e-24
  1.28029521e-24 3.99221047e-24]
 [2.93765604e-26 3.16934938e-26 3.08595310e-26 ... 1.09245384e-24
  1.21853403e-24 3.77014864e-24]
 ...
 [1.05233040e-24 1.14690617e-24 1.09245384e-24 ... 1.51788479e-21
  2.01810671e-21 2.30672932e-20]
 [1.17388485e-24 1.28029521e-24 1.21853403e-24 ... 2.01810671e-21
  2.68942033e-21 3.10988350e-20]
 [3.54500534e-24 3.99221047e-24 3.77014864e-24 ... 2.30672932e-20
  3.10988350e-20 3.85603333e-19]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [155]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 1500 11786 -- 
+ attr: name (v)


In [156]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[755 756 758]
 [809 897 899]
 [ 58  65 138]]
32820


In [157]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[75.         1.3272538  3.6993332]
  [75.00527    1.3272538  3.699471 ]
  [75.09398    1.3272538  3.7639241]]

 [[75.62203   -6.0434117  1.4785457]
  [77.139305  -6.5098743  1.8737592]
  [76.64766   -5.5423326  2.7299964]]

 [[76.01672    7.328478  84.99445  ]
  [76.21391    7.328478  84.93604  ]
  [76.407776   7.328478  84.867386 ]]]
32820


In [158]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [159]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32820, 3)
[[75.03309    1.3272538  3.7209094]
 [76.469666  -6.031873   2.0274336]
 [76.21281    7.328478  84.932625 ]]


#### KNN Tri

In [160]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [161]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32820, 19, 3)


#### calculate e norm matrix

In [162]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32820, 3)
[[0.00527371 0.11403516 0.10965005]
 [1.6358173  1.6938478  1.3823874 ]]


#### Calculate r

In [163]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32820,)

In [164]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32820, 19)
(32820, 19)


In [165]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32820, 19, 5)


#### Calculate f

In [166]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.000000000000003
(1500,)
[0.00066523 0.00066523 0.00066523 ... 0.00066523 0.00066523 0.00066523]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_5324\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [167]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(500, 3, 3)
[[[82.15758   8.760632 87.644714]
  [82.485054  8.760632 87.26128 ]
  [82.79202   8.760632 86.86124 ]]]


In [168]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 461
Number of edges: 896


In [183]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [170]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [171]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[76.761116   3.5589852 97.95591  ]
 [76.730576   4.1574683 97.6653   ]
 [76.78684    2.9401166 98.200676 ]
 ...
 [82.73806   -8.535009  88.938065 ]
 [82.32248   -8.535009  89.332436 ]
 [83.77621    7.138877  -7.037709 ]]
dim sample points :  1500


Distance Matrix

In [172]:
def squared_euclidean_distances(x, y):
    x_squared_sum = np.sum(x**2, axis=1)
    y_squared_sum = np.sum(y**2, axis=1)
    xy_dot_product = np.dot(y, x.T)

    distances = np.outer(y_squared_sum, np.ones_like(x_squared_sum)) - 2 * xy_dot_product + np.outer(np.ones_like(y_squared_sum), x_squared_sum)

    return distances

distances_matrix = squared_euclidean_distances(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 8886.195    6622.0454   8881.104   ... 11822.68    12451.146
  11688.881  ]
 [ 8836.865    6580.012    8830.975   ... 11775.08    12403.346
  11641.8125 ]
 [ 8927.592    6657.3335   8923.324   ... 11860.801   12489.411
  11726.527  ]
 ...
 [ 7369.9067   5342.568    7381.2134  ... 10207.844   10988.525
  10082.812  ]
 [ 7430.977    5393.4536   7442.262   ... 10257.449   11027.641
  10131.901  ]
 [  254.94385   698.59424   245.44092 ...  1321.1785   2256.2622
   1325.6335 ]]


In [173]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [174]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [175]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00018901 0.00018902 0.00018901 ... 0.00018698 0.00019917 0.00018698]


In [176]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 3.9048917022778316


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [177]:
b = np.array(graph)
print("Barycentre b: ", b)

Barycentre b:  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]


In [178]:
b_hat = np.array(extended_graph)
print("B_HAT : ", b_hat)

B_HAT :  [[76.761116   3.5589852 97.95591  ]
 [76.730576   4.1574683 97.6653   ]
 [76.78684    2.9401166 98.200676 ]
 ...
 [82.73806   -8.535009  88.938065 ]
 [82.32248   -8.535009  89.332436 ]
 [83.77621    7.138877  -7.037709 ]]


test matrice des normes euclidiennes entre b et b_hat

In [179]:
from scipy.spatial.distance import cdist, pdist, squareform
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean')
    
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


Distances euclidiennes entre les barycentres b et b_hat :
 [[ 8886.19530457  8836.86391168  8927.59172464 ...  7369.90691817
   7430.97649461   254.94487866]
 [ 6622.04466992  6580.01146999  6657.33236797 ...  5342.56782502
   5393.4534714    698.59457824]
 [ 8881.10325143  8830.97369889  8923.32472298 ...  7381.21400544
   7442.2626367    245.44131284]
 ...
 [11822.67858181 11775.07954668 11860.80033987 ... 10207.84435711
  10257.44883425  1321.17868815]
 [12451.14645374 12403.34460706 12489.41035165 ... 10988.52578332
  11027.64039167  2256.2623569 ]
 [11688.87966907 11641.81079359 11726.52613534 ... 10082.81234338
  10131.90088972  1325.63338961]]


matrice sqeuclidean b

In [180]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b)

[5643.6636 5906.267  5639.44   ... 2723.2537 1636.7125 2716.7126]


In [181]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[  0.           0.66601023   0.66601102 ... 104.2755673   16.2268326
  105.28858983]
 [  0.66601023   0.           1.33106729 ... 103.97229757  16.53340469
  104.98213915]
 [  0.66601102   1.33106729   0.         ... 104.53770236  15.90256201
  105.55377078]
 ...
 [104.2755673  103.97229757 104.53770236 ...   0.          96.08895249
    2.32649574]
 [ 16.2268326   16.53340469  15.90256201 ...  96.08895249   0.
   97.25275122]
 [105.28858983 104.98213915 105.55377078 ...   2.32649574  97.25275122
    0.        ]]


In [182]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    dist_matrixs_1= distances_after - matrix_b
    #dist_matrixs_2= matrix_b - distances_after

    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum(dist_matrixs_1)**2))

    

    return d_S_Ss

result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

b shape: (5999, 3)
b_hat shape: (1500, 3)
p_b_hat shape: (5999,)


ValueError: operands could not be broadcast together with shapes (1500,1500) (5999,) 

## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

